<img src="https://www.python.org/static/img/python-logo.png">

# Welcome to my lessons

---

**Bo Zhang** (NAOC, <mailto:bozhang@nao.cas.cn>) will have a few lessons on python.

- These are **very useful knowledge, skills and code styles** when you use `python` to process astronomical data.
- All materials can be found on [**my github page**](https://github.com/hypergravity/cham_teaches_python).
- **jupyter notebook** (formerly named **ipython notebook**) is recommeded to use

---
These lectures are organized as below:
1. install python
2. basic syntax
3. numerical computing
4. scientific computing
5. plotting
6. astronomical data processing
7. high performance computing
8. version control


# APLpy basics

[aplpyreaddocs](http://aplpy.readthedocs.io/en/stable/#)

### - Introduction

  **APLpy : Astronomical Plotting Library in Python.**

  For convenient fits image or cubes plotting with astronomical coordinates

### - Dependencies

  APLpy is based on **matplotlib**, so any matplotlib originated functions/classes can be    easily adopted.

  For image reprojection or mosaicing purpose, APLpy relies on **Montage**, which is developed by IPAC team. To enable functions like making RGB cube, one should first install Montage package.

[Montage package docs](http://montage.ipac.caltech.edu/docs/)

### - Tips

  For plotting plenty of similar images, one could define a function to set up a set of standard properties (*i.e. def standard_setup( )*), such as axis and tick labels and use this user-defined function instead of the calling the APLpy functions time to time.

  Of course, a different setting by directly calling APLpy functions will overide the settings after calling the user-defined standard setup function.


In [1]:
import aplpy
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt

def standard_setup(fig):
  fig.show_colorscale(cmap='gist_gray')
  fig.tick_labels.set_font(size='large')
  fig.frame.set_linewidth(1)
  fig.frame.set_color('black')
  fig.add_colorbar()
  fig.colorbar.show()
  fig.colorbar.set_location('top')
  fig.colorbar.set_font(size='medium', weight='medium', \
                      stretch='normal', family='sans-serif', \
                      style='normal', variant='normal')
  fig.list_layers()
  fig.axis_labels.hide_y()
  fig.tick_labels.hide_y()
  fig.axis_labels.hide_x()
  fig.tick_labels.hide_x()
  

dirpath = r'/Users/yuxinlin/L3a/0852p302_ab41/'
outpath = r'/Users/yuxinlin/L3a/RGB/'


filepath1= dirpath+r'w4_cut.fits'
filepath2= dirpath+r'w3_cut.fits'
filepath3= dirpath+r'w2_cut.fits'

aplpy.make_rgb_cube([filepath3,filepath2,filepath1], outpath+r'wise234_RGB.fits')

aplpy.make_rgb_image(outpath+r'wise234_RGB.fits',outpath+r'rgb_image_wise234.png', stretch_r='linear', vmid_r=None, pmin_r=10, pmax_r=98,exponent_r=2, vmin_g=None, vmax_g=None, pmin_g=10, pmax_g=98, stretch_g='linear', vmid_g=None, exponent_g=2, vmin_b=None, vmax_b=None, pmin_b=10, pmax_b=98, stretch_b='linear', vmid_b=None)
fits.setval(outpath+r'wise234_RGB.fits','CTYPE3',value='RGB')


# Launch APLpy figure of 2D cube
img = aplpy.FITSFigure(outpath+r'wise234_RGB.fits',dimensions=[0,1],slices=[2],figsize=(10,9))
standard_setup(img)
img.recenter(084.78475,+30.09747,width=0.12,height=0.12)
img.show_rgb(outpath+r'rgb_image_wise234.png')
img.add_scalebar(0.08333,"1 pc",color='white', corner='bottom left')
img.show_contour(r'/Users/yuxinlin/G178/cut/spire_350/G178_final.850.fits',levels=np.linspace(0.08,0.23,5),colors='white')
img.save(outpath+r'rgb_W3_contour.pdf')
plt.show()



Exception: File does not exist : /Users/yuxinlin/L3a/0852p302_ab41/w2_cut.fits

# Healpy 

[aplpyreaddocs](http://aplpy.readthedocs.io/en/stable/#)

### - Introduction

  To take care of fits files of Healpix pixelization schemes (http://healpix.sourceforge.net/downloads.php), including fitsfile io and transformation to standard coordinate systems. 

  This is particularly useful for all-sky survey such as Planck data, or SCUBA2 data release 1.

### - 

In [ ]:
import os
import healpy as hp
from astropy.io import fits
import montage_wrapper as mt

#read the fitsfile
plankCO = hp.read_map('HFI_CompMap_CO-Type3_2048_R1.10.fits')#standard planck survey data from online archive
pixNum = 8192

#view the fitsfile in Cartesian projection
imageData = hp.cartview(plankCO, title='Histogram equalized Ecliptic', 
     unit='K km/s', min=1,max=500, cmap = 'gist_heat_r', 
     norm = 'log', cbar= False, return_projected_map = True, 
     xsize = pixNum)

#fits header editing
hdu = fits.PrimaryHDU(imageData.data)
hdu.header['CTYPE1']  = 'GLON-CAR'                                      
hdu.header['CRVAL1']  =         0.0                           
hdu.header['CDELT1']  =         -360.0/pixNum                        
hdu.header['CRPIX1']  =               pixNum/2.0+0.5                           
hdu.header['CTYPE2']  = 'GLAT-CAR'                                      
hdu.header['CRVAL2']  =         0.0                          
hdu.header['CDELT2']  =           360.0/pixNum                    
hdu.header['CRPIX2']  =              pixNum/4.0+0.5
hdu.header['BUNIT']   = 'K*km/s'
hdu.header['TELESCOP'] = 'Planck'
hdu.writeto('PlanckCO_Type3_car.fits', clobber= True)

#target fitsheader editing
hdrTAN = hdu.header.copy()
hdrTAN['CTYPE1']  = 'GLON-TAN'
hdrTAN['CTYPE2']  = 'GLAT-TAN'

#fitsheader and file output
#use mongtage as the reprojection engine, one could also implement other reprojection packages
if os.path.isfile('TAN.hdr'):
    os.remove('TAN.hdr')

hdrTAN.totextfile('TAN.hdr')

if os.path.isfile('PlanckCO_Type3_tan.fits'):
    os.remove('PlanckCO_Type3_tan.fits')

mt.reproject('PlanckCO_Type3_car.fits', 
    'PlanckCO_Type3_tan.fits', 
    header = 'TAN.hdr')